In [ ]:
# train_distance.py  –  run this on your PC
import numpy as np, tensorflow as tf

# Toy training data: distance(cm)  &  label  (0 = near, 1 = far)
X = np.array([[10],[15],[25],[29],[31],[40],[55]], dtype=np.float32)
y = np.array([[0],[0],[0],[0],[1],[1],[1]], dtype=np.float32)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1,)),
    tf.keras.layers.Dense(1, activation='sigmoid')   # logistic regression
])
model.compile(optimizer='adam', loss='binary_crossentropy')
model.fit(X, y, epochs=200, verbose=0)

# Convert to TFLite-Micro flatbuffer
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("distance_lr.tflite", "wb").write(tflite_model)
print("Model size:", len(tflite_model), "bytes")


Saved artifact at '/tmp/tmpzq9akpjj'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  134931499833424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134931499836688: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model size: 1284 bytes


In [6]:

!pip install scikit-learn micromlgen pandas
!python train_threshold_lr.py
# train_threshold_lr.py
import pandas as pd
from sklearn.linear_model import LogisticRegression
from micromlgen import port             # tiny header generator

data = pd.read_csv('/content/data.csv')
X, y = data[['value']], data['label']
clf = LogisticRegression().fit(X, y)

# produce a single-file C header with predict()
open('model.h', 'w').write(
    port(clf, classmap={0: 'LOW', 1: 'HIGH'}, microoptim=True)
)
print('W =', clf.coef_[0][0], '  B =', clf.intercept_[0])


  Preparing metadata (setup.py) ... done
  Created wheel for micromlgen: filename=micromlgen-1.1.28-py3-none-any.whl size=32152 sha256=f80bf9be6254130b6dded931d1615de45ac021f27b67d14eb7aa1b8ce68905bd
  Stored in directory: /root/.cache/pip/wheels/aa/3a/19/9fd4635f110202e18f3b4bb631e90287ba06eb406999726cb9
Successfully built micromlgen
python3: can't open file '/content/train_threshold_lr.py': [Errno 2] No such file or directory
W = 0.2984243748921464   B = -73.75347759621968
